In [78]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import ipaddress
from user_agents import parse
import httpagentparser

Reading dataset and removing unwanted coloumn

In [221]:
rowDataset = pd.read_csv("../data-processed/imputation-v1.csv")
rowDataset = rowDataset.drop(columns=["URL", "KEYWORDS", "TAXONOMY", "IS_HCP", "CHANNELTYPE", "USERCITY"])
rowDataset.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal


Segregating prediction_dataset whos "USERZIPCODE" is missing

In [222]:
prediction_dataset = rowDataset[(rowDataset["USERZIPCODE"].isna())]
prediction_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
22,1023,Desktop,9,66.249.66.14,821591df-053a-4312-9fed-5a7233063058,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal
24,1025,Mobile,6,107.77.197.130,9bfe23a4-aaf3-482f-be0b-636413863f19,NaN,Mozilla/5.0 (iPhone; CPU iPhone OS 16_4_1 like...,Online Learning Portal
33,1034,Desktop,9,66.249.66.73,8ab58a44-d34b-4e64-aefd-12b735bb6242,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal
40,1041,Desktop,9,66.249.66.3,beb69f72-ec14-4871-9de7-2bc7b6f49a21,NaN,"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Ge...",Online Medical Journal
63,1064,Desktop,2,166.196.89.101,f3bb0700-be6c-4a51-b756-488d7b881e9a,NaN,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
...,...,...,...,...,...,...,...,...
113835,114836,Desktop,9,66.249.66.75,43b6ce21-4589-4d9a-82d5-0e736b0f5667,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal
113848,114849,Desktop,9,66.249.66.85,0e9490e5-bb3b-412f-b538-172b1a223408,NaN,"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Ge...",Online Medical Journal
113884,114885,Mobile,2,172.58.62.230,0f4a86c3-ffe5-4796-aab0-804fd43c9623,NaN,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal
113905,114906,Desktop,9,66.249.66.33,37bff2a8-34cd-4c30-bd9f-75e0c98044e2,NaN,"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Ge...",Online Medical Journal


Segregating training, testing and validating dataset

In [259]:
useful_dataset = rowDataset[~(rowDataset["USERZIPCODE"].isna())]
useful_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal
...,...,...,...,...,...,...,...,...
113932,114933,Desktop,2,68.82.97.126,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal
113933,114934,Desktop,2,104.172.11.109,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,914014213.0,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,Online Medical Journal
113934,114935,Desktop,7,174.21.94.113,9c36e90c-caa7-4323-8431-fb84f8293972,98105.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
113935,114936,Mobile,2,69.253.129.131,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal


Checking missing values in dataset

In [260]:
useful_dataset.isna().sum()

ID                 0
DEVICETYPE         0
PLATFORM_ID        0
BIDREQUESTIP       0
USERPLATFORMUID    4
USERZIPCODE        0
USERAGENT          1
PLATFORMTYPE       0
dtype: int64

In [261]:
useful_dataset.dropna(inplace=True)

/tmp/ipykernel_516/2009984717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_dataset.dropna(inplace=True)


Geting 'undefined' values indexes

In [262]:
encoder_deviceType = LabelEncoder()
encoder_deviceType.fit(['Desktop', 'Mobile', 'Tablet', 'Unknown'])
useful_dataset["DEVICETYPE"] = encoder_deviceType.transform(useful_dataset["DEVICETYPE"])


/tmp/ipykernel_516/3785491334.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_dataset["DEVICETYPE"] = encoder_deviceType.transform(useful_dataset["DEVICETYPE"])


In [263]:
useful_dataset.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,0,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
1,1002,0,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
2,1003,0,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
3,1004,0,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
4,1005,1,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal


Testing code

In [11]:
ip_address_int = int(ipaddress.IPv4Address("170.173.0.22"))
ip_address_bin = bin(ip_address_int)[2:].zfill(32)

Converting unique IP address to int

In [264]:
def convert_to_int(value):
    return int(ipaddress.IPv4Address(value))

useful_dataset["BIDREQUESTIP"] = useful_dataset["BIDREQUESTIP"].apply(convert_to_int)

/tmp/ipykernel_516/2307204237.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_dataset["BIDREQUESTIP"] = useful_dataset["BIDREQUESTIP"].apply(convert_to_int)


Format change of zipcode value

In [265]:
useful_dataset["USERZIPCODE"] = useful_dataset["USERZIPCODE"].apply(int)

/tmp/ipykernel_516/1982257390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_dataset["USERZIPCODE"] = useful_dataset["USERZIPCODE"].apply(int)


In [266]:
useful_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal
...,...,...,...,...,...,...,...,...
113932,114933,0,2,1146249598,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal
113933,114934,0,2,1756105581,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,914014213,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,Online Medical Journal
113934,114935,0,7,2920636017,9c36e90c-caa7-4323-8431-fb84f8293972,98105,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
113935,114936,1,2,1174241667,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal


In [267]:
useful_dataset.loc[useful_dataset["USERPLATFORMUID"]=="undefined"]
useful_dataset = useful_dataset.drop(index=[60591, 98237])

In [268]:
useful_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal
...,...,...,...,...,...,...,...,...
113932,114933,0,2,1146249598,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal
113933,114934,0,2,1756105581,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,914014213,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,Online Medical Journal
113934,114935,0,7,2920636017,9c36e90c-caa7-4323-8431-fb84f8293972,98105,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal
113935,114936,1,2,1174241667,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal


In [156]:
platformId = useful_dataset["USERPLATFORMUID"]

In [157]:
platformId = list(platformId)

In [159]:
def sliptId(value):
    return value.split("-")

In [160]:
result = [sliptId(index) for index in platformId]

In [162]:
platform_df = pd.DataFrame(result, columns=["A", "B", "C","D", "F"])

In [163]:
platform_df["ID"] = useful_dataset["ID"]

In [176]:
platform_df.head()

,A,B,C,D,F,ID
0,6974dcaa,f932,480e,9fb5,c52e20e1393a,1001.0
1,c12f3f8f,8fcf,484a,90e1,1ac04db8cdcf,1002.0
2,a698de4b,e200,46dd,b5fb,40402175ae18,1003.0
3,45967533,75c8,4fbd,a00c,e6ff20447aaa,1004.0
4,a17e25be,532d,4cf5,b916,9308c8c3961f,1005.0


In [177]:
def hex_to_int(hex_value):
    return int(hex_value, 16)

In [184]:
platform_df["F_hex"] = platform_df["F"].apply(hex_to_int)

In [194]:
platform_int_df = platform_df[["A_hex", "B_hex", "C_hex", "D_hex", "F_hex"]]

In [198]:
platform_int_df.to_csv("../data-processed/uid.csv", index=False)

In [201]:
useful_dataset[["UID_A", "UID_B", "UID_C", "UID_D", "UID_F"]] = platform_df[["A_hex", "B_hex", "C_hex", "D_hex", "F_hex"]]

In [203]:
useful_dataset = useful_dataset.drop(columns=["USERPLATFORMUID"])

In [235]:
useful_dataset["PLATFORMTYPE"].unique()

array(['Online Medical Journal', 'Online Learning Portal',
       'Medical News Platform', 'Electronic Health Record'], dtype=object)

In [269]:
encoder_platformType = LabelEncoder()
encoder_platformType.fit(['Online Medical Journal', 'Online Learning Portal','Medical News Platform', 'Electronic Health Record'])
useful_dataset["PLATFORMTYPE"] = encoder_platformType.transform(useful_dataset["PLATFORMTYPE"])

In [271]:
useful_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,3
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,3
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,3
...,...,...,...,...,...,...,...,...
113932,114933,0,2,1146249598,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3
113933,114934,0,2,1756105581,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,914014213,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,3
113934,114935,0,7,2920636017,9c36e90c-caa7-4323-8431-fb84f8293972,98105,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,3
113935,114936,1,2,1174241667,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,3


In [277]:
user_agent_series = useful_dataset[["ID", "USERAGENT"]].copy()
user_agent_series

,ID,USERAGENT
0,1001,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,1002,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...
2,1003,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...
3,1004,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,1005,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...
...,...,...
113932,114933,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...
113933,114934,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
113934,114935,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...
113935,114936,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...


Approach #1 : Using user_agents.parse function

In [58]:
def get_user_details(value):
    user_details = parse(value)
    return user_details


user_details = user_agent_series.apply(get_user_details)

browers = []
oses = []
devies = []

for ua in user_details:
    browers.append(ua.browser.family)
    oses.append(ua.os.family)
    devies.append(ua.device.family)

user_agent_df = pd.DataFrame({"BROWER":browers, "OS": oses, "DEVICE":devies})

user_agent_df.to_csv("../data-processed/user-agent-a1.csv", index=False)

Approach #2 : Using httpagentparser function

In [293]:
def get_user_agent_details_a2(value):
    return httpagentparser.detect(value)

user_details = user_agent_series["USERAGENT"].apply(get_user_agent_details_a2)

In [297]:
user_details[0]

{'platform': {'name': 'Windows', 'version': '10'},
 'os': {'name': 'Windows', 'version': '10'},
 'bot': False,
 'browser': {'name': 'Chrome', 'version': '112.0.0.0'}}

In [298]:
def get_platform_details(value, key):
    try:
        if key == "bot":
            return value[key]
        return value[key]["name"]
    except:
        return None


platform_temp = []
os_temp = []
bot_temp = []
browser_temp = []
dist_temp = []
for us in user_details.index:
    platform_temp.append(get_platform_details(user_details[us], "platform"))
    os_temp.append(get_platform_details(user_details[us], "os"))
    bot_temp.append(get_platform_details(user_details[us], "bot"))
    browser_temp.append(get_platform_details(user_details[us], "browser"))
    dist_temp.append(get_platform_details(user_details[us], "dist"))


In [354]:
len(bot_temp)

109338

In [340]:
details = pd.DataFrame( {"ID":user_agent_series["ID"], "PLATFORM": platform_temp,"BOT": bot_temp, "BROWSER": browser_temp, "DIST": dist_temp})

In [358]:
details

,ID,PLATFORM,BOT,BROWSER,DIST
0,1001,Windows,False,Chrome,None
1,1002,Mac OS,False,Safari,None
2,1003,Mac OS,False,Safari,None
3,1004,Windows,False,ChromiumEdge,None
4,1005,iOS,False,Safari,iPhone
...,...,...,...,...,...
113932,114933,Mac OS,False,Safari,None
113933,114934,Windows,False,Chrome,None
113934,114935,Mac OS,False,Chrome,None
113935,114936,iOS,False,Safari,iPhone


In [367]:
useful_dataset = pd.merge(useful_dataset, details, on='ID', how='inner')

In [370]:
useful_dataset = useful_dataset.drop(columns=["USERAGENT"])

In [371]:
useful_dataset.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,PLATFORMTYPE,PLATFORM,BOT,BROWSER,DIST
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,3,Windows,False,Chrome,None
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,3,Mac OS,False,Safari,None
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,3,Mac OS,False,Safari,None
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,3,Windows,False,ChromiumEdge,None
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,3,iOS,False,Safari,iPhone


In [356]:
details["DIST"].isna().sum()

75048

In [ ]:
for index in details.index:
    if details.loc[index]["PLATFORM"] == details.loc[index]["OS"]:
        continue
    else:
        print(index)

In [339]:
details["OS"].value_counts()

Windows      48292
Macintosh    24789
iOS          24585
Linux        10322
ChromeOS      1154
Name: OS, dtype: int64

In [343]:
details["PLATFORM"].value_counts()

Windows      48292
Mac OS       24789
iOS          24585
Android       9681
 ChromeOS     1154
Linux          783
Name: PLATFORM, dtype: int64

In [310]:
for column in details.columns:
    print(f"{column} ==> \n{details[column].unique()}")

ID ==> 
[  1001   1002   1003 ... 114935 114936 114937]
PLATFORM ==> 
['Windows' 'Mac OS' 'iOS' 'Android' ' ChromeOS' 'Linux' None]
OS ==> 
['Windows' 'Macintosh' 'iOS' 'Linux' 'ChromeOS' None]
BOT ==> 
[False True None]
BROWSER ==> 
['Chrome' 'Safari' 'ChromiumEdge' 'Firefox' 'ChromeiOS' None 'MSEdge'
 'Opera' 'BingBot' 'FacebookExternalHit']
DIST ==> 
[None 'iPhone' 'IPad' 'Android' 'Ubuntu']


In [252]:
len(details)

109338

In [342]:
details.to_csv("../data-processed/user-agent-a2.csv", index=False)

In [248]:
useful_dataset = useful_dataset.drop(columns=["USERAGENT"])

In [288]:
useful_dataset

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,USERAGENT,PLATFORMTYPE
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,3
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,3
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,3
...,...,...,...,...,...,...,...,...
113932,114933,0,2,1146249598,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3
113933,114934,0,2,1756105581,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,914014213,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,3
113934,114935,0,7,2920636017,9c36e90c-caa7-4323-8431-fb84f8293972,98105,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,3
113935,114936,1,2,1174241667,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,3


In [250]:
useful_dataset["PLATFORM_NAME"] = details["PLATFORM"]
useful_dataset["OS"] = details["OS"]
useful_dataset["BOT"] = details["BOT"]
useful_dataset["BROWSER"] = details["BROWSER"]
useful_dataset["DIST"] = details["DIST"]

In [253]:
useful_dataset.drop(columns=["PLATFORM_NAME", "OS", "BOT", "BROWSER", "DIST"])

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,PLATFORMTYPE
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,3
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,3
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,3
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,3
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,3
...,...,...,...,...,...,...,...
113932,114933,0,2,1146249598,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,19120,3
113933,114934,0,2,1756105581,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,914014213,3
113934,114935,0,7,2920636017,9c36e90c-caa7-4323-8431-fb84f8293972,98105,3
113935,114936,1,2,1174241667,afc27760-31f7-4f6e-aa08-90d8a96525d0,19808,3


In [377]:
useful_dataset.to_csv("../data-processed/imputation-v2.csv", index=False)

In [375]:
import hashlib

str2hash = '2863464470'
result = hashlib.md5(str2hash.encode())
print(result.hexdigest())

706f7a12ea6098e4d7873ccee7bd88d7
